# Lab 13 Part 2: FastAPI Application Setup and WebSocket Integration

**Duration:** 10 minutes  

## Learning Objectives

- Initialize FastAPI application with template support
- Configure WebSocket connection manager for real-time features
- Set up connection handling and broadcasting
- Configure CORS and middleware settings

## Import Required Dependencies

Ensure all necessary libraries are imported for this notebook.

In [ ]:
from fastapi import FastAPI, Request, WebSocket, WebSocketDisconnect
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import json
import asyncio
from datetime import datetime, timedelta
from typing import List, Dict, Any
import uvicorn
from neo4j import GraphDatabase
import os

print("✓ Libraries imported successfully")

## Initialize FastAPI Application

Create the main FastAPI application instance with configuration for the insurance web platform.

In [ ]:
# Create FastAPI application
app = FastAPI(
    title="Neo4j Insurance Web Application",
    description="Interactive insurance platform with real-time features",
    version="1.0.0"
)

# Setup templates and static files
templates = Jinja2Templates(directory="templates")

print("✓ FastAPI application initialized")

## WebSocket Connection Manager

Create a WebSocket manager to handle real-time connections, broadcasting, and user-specific messaging.

In [ ]:
class WebSocketManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []
        self.user_connections: Dict[str, WebSocket] = {}
    
    async def connect(self, websocket: WebSocket, user_id: str = None):
        await websocket.accept()
        self.active_connections.append(websocket)
        if user_id:
            self.user_connections[user_id] = websocket
    
    def disconnect(self, websocket: WebSocket, user_id: str = None):
        if websocket in self.active_connections:
            self.active_connections.remove(websocket)
        if user_id and user_id in self.user_connections:
            del self.user_connections[user_id]
    
    async def send_personal_message(self, message: str, websocket: WebSocket):
        await websocket.send_text(message)
    
    async def broadcast(self, message: str):
        for connection in self.active_connections:
            try:
                await connection.send_text(message)
            except:
                # Remove broken connections
                if connection in self.active_connections:
                    self.active_connections.remove(connection)
    
    async def send_to_user(self, user_id: str, message: str):
        if user_id in self.user_connections:
            try:
                await self.user_connections[user_id].send_text(message)
            except:
                del self.user_connections[user_id]

websocket_manager = WebSocketManager()

print("✓ WebSocket manager configured")

## WebSocket Connection Testing

Verify the WebSocket manager is properly initialized and ready for connections.

In [ ]:
# Verify WebSocket manager status
print(f"Active connections: {len(websocket_manager.active_connections)}")
print(f"User connections: {len(websocket_manager.user_connections)}")
print("✓ WebSocket manager ready for real-time connections")

## Summary

In this notebook, you:
- Initialized the FastAPI application with metadata
- Set up template rendering support
- Created a WebSocket connection manager
- Implemented connection, disconnection, and broadcasting functionality
- Verified the WebSocket manager is ready

Next: Proceed to notebook 03 to implement dashboard routes and customer APIs.